In [1]:
import sys
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *

In [3]:
class UI(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()

    def init_ui(self):
        self.resize(1500, 750)
        self.center()
        self.setWindowTitle('Clustering agorithm')
        
        label1 = QLabel('<h1>Welcome to our app!</h1>')
        label1.setAlignment(Qt.AlignHCenter)
        
        label2 = QLabel('<h3>Upload a file to run the t-sne algorithm on</h3>')
        label2.setAlignment(Qt.AlignCenter)

        
        btn = QPushButton("Open File", self)
        btn.setToolTip('Click here to upload a file')
        btn.setFixedSize(100, 40)
        
        helpbtn = QPushButton("Help", self)
        helpbtn.setToolTip('Click here to read instructions')
        helpbtn.setFixedSize(100, 40)
        
        layout = QVBoxLayout()
        layout.addWidget(label1)
        layout.addStretch()
        layout.addWidget(label2)
        layout.addWidget(btn, alignment=Qt.AlignCenter)
        layout.addWidget(helpbtn, alignment=Qt.AlignCenter)
        layout.addStretch()

        self.setLayout(layout)
        btn.clicked.connect(self.open)
        helpbtn.clicked.connect(self.open_instructions_window)
        
        self.show()

    def center(self):
        # geometry of the main window
        qr = self.frameGeometry()

        # center point of screen
        cp = QDesktopWidget().availableGeometry().center()

        # move rectangle's center point to screen's center point
        qr.moveCenter(cp)

        # top left of rectangle becomes top left of window centering it
        self.move(qr.topLeft())
        
    def open(self):
        path = QFileDialog.getOpenFileName(self, 'Open a file', '',
                                        'All Files (*.*)')
        if path != ('', ''):
            print("File path : "+ path[0])
        self.w = InstructionsWindow()
        self.w.show()
            
    def open_instructions_window(self, checked):
        self.w = ClusteringAlgorithmWindow()
        self.w.show()
        
class InstructionsWindow(QWidget):
  
    def __init__(self):
        super().__init__()
        self.init_ui()
        
    def init_ui(self):
        self.resize(1500, 750)
        self.center()
        self.setWindowTitle('Clustering agorithm instructions')
        
        label1 = QLabel('<h1>Read the instructions on how to use the clustering algorithm app</h1>')
        label1.setAlignment(Qt.AlignHCenter)
        
        label2 = QLabel('<h3>1.</h3>')
        label2.setAlignment(Qt.AlignHCenter) 
        
        label3 = QLabel('<h3>2.</h3>')
        label3.setAlignment(Qt.AlignHCenter) 
        
        label4 = QLabel('<h3>3.</h3>')
        label4.setAlignment(Qt.AlignHCenter) 
        
        layout = QVBoxLayout()
        layout.addWidget(label1)
        layout.addWidget(label2)
        layout.addWidget(label3)
        layout.addWidget(label4)

        self.setLayout(layout)
        
        self.show()
        
    def center(self):
        # geometry of the main window
        qr = self.frameGeometry()

        # center point of screen
        cp = QDesktopWidget().availableGeometry().center()

        # move rectangle's center point to screen's center point
        qr.moveCenter(cp)

        # top left of rectangle becomes top left of window centering it
        self.move(qr.topLeft())
        
class ClusteringAlgorithmWindow(QWidget):
  
    def __init__(self):
        super().__init__()
        self.init_ui()
        
    def init_ui(self):
        self.resize(1500, 750)
        self.center()
        self.setWindowTitle('Clustering agorithm outputs')
        
        label1 = QLabel('<h1>The outputs of the algorithm will show up here</h1>')
        label1.setAlignment(Qt.AlignHCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(label1)

        self.setLayout(layout)
        
        self.show()
        
    def center(self):
        # geometry of the main window
        qr = self.frameGeometry()

        # center point of screen
        cp = QDesktopWidget().availableGeometry().center()

        # move rectangle's center point to screen's center point
        qr.moveCenter(cp)

        # top left of rectangle becomes top left of window centering it
        self.move(qr.topLeft())

if __name__ == "__main__":
    app = QApplication(sys.argv)
    ex = UI()
    sys.exit(app.exec_())

AttributeError: 'UI' object has no attribute 'center'